# 1. check the grammar: 

In [174]:
import language_check
from nltk.tokenize import word_tokenize

def grammar_score(doc):
    lc_tool=language_check.LanguageTool('en-US')
    
    matchs=lc_tool.check(doc)
    
    mCount=len(matchs)
    word_cnt=len(word_tokenize(doc))
    
    score=0
    if mCount >=5: 
        score=0
    else : 
        score=(word_cnt-mCount)/word_cnt
    #print(matchs)
    errorlist=[]
    error={}
    for t in matchs:
        errorlist.append(str(t))
        #error['offset']=t['offset']
        #error['msg']=t['msg']
        #errorlist.append(error)
        
    
    msg='This paragraph contains at lest {} program errors: \n{}'.format(len(matchs), '-'.join(errorlist))
    print(msg)
    return (score,msg)



In [175]:
text=u'Population is made up of monthly meetings bonus fro DEC to MAY each year, therefore a sample size of two considered adequate'

In [173]:
grammar_score(text)

This paragraph contains at lest 2 program errors: 
Line 1, column 34, Rule ID: POSSESSIVE_APOSTROPHE[1]
Message: Possible typo: apostrophe is missing. Did you mean 'meetings'' or 'meeting's'?
Suggestion: meetings'; meeting's
Population is made up of monthly meetings bonus fro DEC to MAY each year, therefo...
                                 ^^^^^^^^-Line 1, column 49, Rule ID: FOR_FRO[1]
Message: Did you mean 'for' or 'from'?
Suggestion: for; from
...on is made up of monthly meetings bonus fro DEC to MAY each year, therefore a sampl...
                                           ^^^


(0.9130434782608695,
 "This paragraph contains at lest 2 program errors: \nLine 1, column 34, Rule ID: POSSESSIVE_APOSTROPHE[1]\nMessage: Possible typo: apostrophe is missing. Did you mean 'meetings'' or 'meeting's'?\nSuggestion: meetings'; meeting's\nPopulation is made up of monthly meetings bonus fro DEC to MAY each year, therefo...\n                                 ^^^^^^^^-Line 1, column 49, Rule ID: FOR_FRO[1]\nMessage: Did you mean 'for' or 'from'?\nSuggestion: for; from\n...on is made up of monthly meetings bonus fro DEC to MAY each year, therefore a sampl...\n                                           ^^^")

# 2 Similarity check: 

In [87]:
from nltk.tokenize import word_tokenize
import json

In [96]:
def similarity_check(doc):
    text=doc.lower()
    word_cnt=len(word_tokenize(text))
    with open('population1.json') as f:
        data=json.load(f)
    match_cnt=0
    keywords={}
    for example in data['rasa_nlu_data']['common_examples']:
        ents={}
        entlist=[]
        for entitem in example['entities']: 
            t=entitem['value']
            key=entitem['entity']
            if text.find(t.lower())>=0:
                #match
                match_cnt=match_cnt+1
                if key in keywords : 
                    valuelist=keywords.get(key)
                    valuelist.append(t)
                else: 
                    valuelist=[]
                    valuelist.append(t)
                    keywords[key]=valuelist
    for k in keywords.keys(): 
        s=set(keywords[k])
        keywords[k]=s
        
    y=0
    if 'source_data' in keywords: 
        if 'frequency' in keywords or 'control_verb' in keywords: 
            y=1
        else: 
            y=0.5
    else: 
        y=0    

    return (y,str(keywords))

In [97]:
similarity_check('The monthly tracker and action logs for the previous 12 months.')

(1,
 "{'frequency': {'monthly', ' monthly'}, 'source_data': {'tracker and action logs'}}")

# 3. Classification

In [74]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
import spacy
nlp=spacy.load('en')
from rasa_nlu.model import Metadata, Interpreter

In [79]:
def classification_check(doc): 
    interpreter=Interpreter.load('/Users/liver/jupyter/audit_pop1/default/model_20180918-232732/')
    result=interpreter.parse(doc)
    return('The paragraph is classified as {} in the confidence level of {}'.format(result['intent']['name'],result['intent']['confidence']))

In [80]:
text='Population is the 10 FCC staff in HMLG. Job descriptions for FCC staff are reviewed annually and it was proposed to test whether job descriptions include adequate reference to control of financial crime risk.'
classification_check(text)

'The paragraph is classified as good1 in the confidence level of 0.8629302579651378'

# 4. NER 

In [73]:
import spacy
def ner_extraction(doc): 
    output_dir='/Users/liver/jupyter/audit-pop1-ner-train'
    nlp2=spacy.load(output_dir)
    doc=nlp2(doc)
    #ents=[(ent.text, ent.label_) for ent in doc.ents]
    entlist={}
    
    for ent in doc.ents: 
        txt=ent.text
        lab=ent.label_
        if lab in entlist : 
            valuelist=entlist.get(lab)
            valuelist.append(txt)
        else: 
            valuelist=[]
            valuelist.append(txt)
            entlist[lab]=valuelist
    #generate score
    iScore=0
    y=0
    if 'source_data' in entlist: 
        if 'frequency' in entlist or 'control_verb' in entlist: 
            y=1
        else: 
            y=0.5
    else: 
        y=0
    
    return(y,"entitiys are found in {} type(s):{}".format(len(entlist), str(entlist)))
    


#pupulation 1 
text='The monthly tracker and action logs for the previous 12 months.'
ner_extraction(text)



(1,
 "entitiys are found in 2 type(s):{'frequency': ['monthly'], 'source_data': ['tracker and action logs']}")

In [62]:
t={'t1':'v1','t2':'v2', 't2':'v3'}
entlist={}

for ti in t.keys():
    valuelist=[]
    entlist[ti]=valuelist
for ti in t: 
    
    
print(str(entlist))

{'t1': [], 't2': []}
